In [2]:
# to be able to import package
import sys; sys.path.append("../")

In [4]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from sklearn.linear_model import LogisticRegression
from markovml.markovml import MarkovReward
from markovml.utils.models_ext import DecisionRules


## A simple Markov reward process
Two states: alive and dead

In [5]:
mrp = MarkovReward(n_states=2, n_features=2)

Restricted license - for non-production use only - expires 2026-11-23


Adding an ML model to predict mortality (prob of 1 -> 2)

In [6]:
# random data
X = np.random.rand(10000, 2)  # 2 features
y = np.random.randint(0, 2, 10000)  # 0 or 1

# fit a logistic regression model
model = LogisticRegression().fit(X, y)

# add the model to the mrp
mrp.add_ml_model(model)

Setting the parameters of the Markov process

In [7]:
mrp.set_pi([1, 0]) # everyone starts alive
mrp.set_P([[1 - mrp.ml_outputs[0][0], mrp.ml_outputs[0][0]], [0, 1]]) # linked to ML
mrp.set_r([1, 0]) # reward is 1 if alive, 0 if dead

Add constraint: mortality at least 0.1

In [8]:
mrp.add_constraint(mrp.P[0, 1] >= 0.1)

Optimize!

In [9]:
mrp.optimize(sense="max")

{'status': 'optimal',
 'objective': 1.0,
 'values': {'pi': [1.0, 0.0],
  'P': [[-0.0, 1.0], [0.0, 1.0]],
  'r': [1.0, 0.0],
  'v': [1.0, -0.0],
  'features': [0.0, 1.5803794978243257],
  'ml_outputs': [[1.0]]}}

## Markov reward process with `Decision Rules`


In [10]:
mrp = MarkovReward(n_states=2, n_features=2, pi=[1, 0], r=[1, 0])

Simple decision rules: age and sex predict mortality

In [11]:
dr = DecisionRules(features=['sex', 'age'])
dr.fit([
    "if sex == 0 and age < 18 then 0.2",
    "if sex == 1 and age < 18 then 0.1",
    "if sex == 0 and age >= 18 then 0.1",
    "if sex == 1 and age >= 18 then 0.2",
    "else 0.0"
])

In [12]:
mrp.add_ml_model(dr)
mrp.set_P([[1 - mrp.ml_outputs[0][0], mrp.ml_outputs[0][0]], [0, 1]])
mrp.add_feature_constraint(mrp.features[1] >= 65) # age >= 65

Optimize!

In [13]:
mrp.optimize(sense="min")

{'status': 'optimal',
 'objective': 4.464285714285713,
 'values': {'pi': [1.0, -0.0],
  'P': [[0.8, 0.2], [-0.0, 1.0]],
  'r': [1.0, -0.0],
  'v': [4.464285714285714, -0.0],
  'features': [0.0, 65.0],
  'ml_outputs': [[0.2]]}}